In [4]:
from ase.io import iread, read
import os
import re
import numpy as np
from pathlib import Path
import pymatgen as pm
from colabfit.tools.configuration import AtomicConfiguration
from colabfit.tools.database import generate_string
import os
import glob
import numpy as np
from pathlib import Path
from ase.io import iread, read
import itertools
from collections import Counter
from colabfit.tools.configuration import AtomicConfiguration
from tqdm import tqdm
from vastdb.session import Session

In [ ]:
import pyspark.sql.functions as sf

newoc = spark.table("ndb.`colabfit-prod`.prod.po")
oc20_20m_id = "DS_otx1qc9f3pm4_0"
newoc = newoc.filter(sf.col("dataset_id") == "DS_otx1qc9f3pm4_0")
oldoc = spark.table("ndb.colabfit.dev.po_oc_test")
oldoc = oldoc.filter(sf.col("dataset_id") == "DS_otx1qc9f3pm4_0")
missing = oldoc.join(newoc, oldoc.id == newoc.id, how="left_anti")
missing.write.csv("oc20_missing.csv", header=True)

In [ ]:
ds = spark.table("ndb.`colabfit-prod`.prod.ds")
nprops_null = ds.filter(sf.col("nproperty_objects").isNull())
nprops_null.select("name", "id").show(20, False)
ids = nprops_null.select("id").collect()
[x["id"] for x in ids]
names = nprops_null.select("name").collect()
[x["name"] for x in names]
ids = [
    "DS_e471qdt7c6db_0",
    "DS_5h3810yhu4wj_0",
    "DS_jz1q9juw7ycj_0",
    "DS_mvuwxu67yrdy_0",
    "DS_la08goe2lz0g_0",
    "DS_0j2smy6relq0_0",
    "DS_dhe9aqs9q1wf_0",
    "DS_1nbddfnjxbjc_0",
    "DS_14m394gnh3ae_0",
    "DS_tat5i46x3hkr_0",
    "DS_dbgckv1il6v7_0",
    "DS_gpsibs9f47k4_0",
]
names = [
    "JARVIS_QE_TB",
    "Fe_nanoparticles_PRB_2023",
    "JARVIS_QM9_STD_JCTC",
    "JARVIS_HOPV",
    "discrepencies_and_error_metrics_NPJ_2023_vacancy_re_testing_set",
    "NENCI-2021",
    "discrepencies_and_error_metrics_NPJ_2023_interstitial_re_testing_set",
    "JARVIS_SNUMAT",
    "Silica_NPJCM_2022",
    "JARVIS_QM9-DGL",
    "JARVIS_Polymer-Genome",
    "JARVIS_Materials_Project_2020",
]
nprops_null.write.csv("datasets_null_nproperty_objects_2024_09_10", header=True)

In [ ]:
# pos = spark.table("ndb.`colabfit-prod`.prod.co")
cos = spark.table("ndb.colabfit.dev.co_remove_dataset_ids")
for id in ids:
    first_count = cos.filter(sf.col("dataset_ids") == f"['{id}']").count()
    print(f"{id} only: {first_count}")
    second_count = cos.filter(sf.col("dataset_ids").contains(id)).count()
    print(f"{id} with others: {second_count}")

In [ ]:
# TRY USING THE SDK TO SEE WHETHER PROJECTIONS ARE FASTER THAN USING SPARK

from vastdb.session import Session
from dotenv import load_dotenv
import os

load_dotenv()
endpoint = os.getenv("VAST_DB_ENDPOINT")
secret = os.getenv("VAST_DB_SECRET")
access = os.getenv("VAST_DB_ACCESS")
session = Session(endpoint=endpoint, access=access, secret=access)
table_split = "ndb.colabfit.dev.co_remove_dataset_ids".split(".")
with session.transaction() as tx:
    table = tx.bucket("colabfit").schema("dev").table("co_remove_dataset_ids_stage2")
    rec_batch_reader = table.select(
        predicate=table.dataset_ids.contains("DS_e471qdt7c6db_0")
    )
    rec_batch = rec_batch_reader.read_next_batch()

In [ ]:
ds.filter(sf.col("id") == "DS_qxd7wv9yabtp_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# discrepencies_and_error_metrics_NPJ_2023_vacancy_enhanced_training_set 218 pos
# Have to remove pos as well as cos

ds.filter(sf.col("id") == "DS_q6e3bvq4y67a_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# discrepencies_and_error_metrics_NPJ_2023_enhanced_validation_set 50 pos
# Have to remove pos as well as cos

ds.filter(sf.col("id") == "DS_nublbp38wse0_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# discrepencies_and_error_metrics_NPJ_2023_interstitial_enhanced_training_set 218 pos
# Entangled with DS_q6e3bvq4y67a_0 above


ds.filter(sf.col("id") == "DS_la08goe2lz0g_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# discrepencies_and_error_metrics_NPJ_2023_vacancy_re_testing_set no pos
# Entangled with DS_q6e3bvq4y67a_0 above

ds.filter(sf.col("id") == "DS_dhe9aqs9q1wf_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# discrepencies_and_error_metrics_NPJ_2023_interstitial_re_testing_set no pos
# Entangled with DS_nublbp38wse0_0 above
ds.filter(sf.col("id") == "DS_14m394gnh3ae_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# Silica_NPJCM_2022 no pos
# entangled with DS_u9wd92plbetq_0 below

ds.filter(sf.col("id") == "DS_gpsibs9f47k4_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# JARVIS_Materials_Project_2020 no pos
# entangled with DS_5ar73nonq6l1_0 and DS_9yr94hhj1k33_0 below


ds.filter(sf.col("id") == "DS_u9wd92plbetq_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# Si_PRX_GAP 2232 pos
ds.filter(sf.col("id") == "DS_5ar73nonq6l1_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# JARVIS_MEGNet2 133407 pos
ds.filter(sf.col("id") == "DS_9yr94hhj1k33_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# JARVIS_MEGNet 69227 pos

#############################################################################################
# Remove all instances of the below cos
ds.filter(sf.col("id") == "DS_e471qdt7c6db_0").select(
    "id", "name", "nproperty_objects"
).show()
# Jarvis qetb no property objects: only duplicates of dataset id
# remove all singles and multiples
ds.filter(sf.col("id") == "DS_5h3810yhu4wj_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# Fe_nanoparticles_PRB_2023 no pos
ds.filter(sf.col("id") == "DS_jz1q9juw7ycj_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# JARVIS_QM9_STD_JCTC no pos
ds.filter(sf.col("id") == "DS_mvuwxu67yrdy_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# JARVIS_HOPV no pos
ds.filter(sf.col("id") == "DS_0j2smy6relq0_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# NENCI-2021 no pos

ds.filter(sf.col("id") == "DS_tat5i46x3hkr_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# JARVIS_QM9-DGL no pos
ds.filter(sf.col("id") == "DS_dbgckv1il6v7_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# JARVIS_Polymer-Genome no pos


ds.filter(sf.col("id") == "DS_1nbddfnjxbjc_0").select(
    "id", "name", "nproperty_objects"
).show(truncate=False)
# JARVIS_SNUMAT no pos

In [ ]:
# #output from above:
# DS_e471qdt7c6db_0 dataset ids: [Row(dataset_ids="['DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0']"),
#                                 Row(dataset_ids="['DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0']"),
#                                 Row(dataset_ids="['DS_e471qdt7c6db_0']"),
#                                 Row(dataset_ids="['DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0']")]
# DS_5h3810yhu4wj_0 dataset ids: [Row(dataset_ids="['DS_5h3810yhu4wj_0']")]
# DS_jz1q9juw7ycj_0 dataset ids: [Row(dataset_ids="['DS_jz1q9juw7ycj_0']")]
# DS_mvuwxu67yrdy_0 dataset ids: [Row(dataset_ids="['DS_mvuwxu67yrdy_0']")]

# DS_la08goe2lz0g_0 dataset ids: [Row(dataset_ids="['DS_qxd7wv9yabtp_0', 'DS_la08goe2lz0g_0']"), Row(dataset_ids="['DS_q6e3bvq4y67a_0', 'DS_la08goe2lz0g_0']"), Row(dataset_ids="['DS_la08goe2lz0g_0']")]
# DS_0j2smy6relq0_0 dataset ids: [Row(dataset_ids="['DS_0j2smy6relq0_0']")]
# DS_dhe9aqs9q1wf_0 dataset ids: [Row(dataset_ids="['DS_nublbp38wse0_0', 'DS_dhe9aqs9q1wf_0']"), Row(dataset_ids="['DS_dhe9aqs9q1wf_0']"), Row(dataset_ids="['DS_q6e3bvq4y67a_0', 'DS_dhe9aqs9q1wf_0']")]
# DS_1nbddfnjxbjc_0 dataset ids: [Row(dataset_ids="['DS_1nbddfnjxbjc_0']")]
# DS_14m394gnh3ae_0 dataset ids: [Row(dataset_ids="['DS_14m394gnh3ae_0']"), Row(dataset_ids="['DS_u9wd92plbetq_0', 'DS_14m394gnh3ae_0']")]
# DS_tat5i46x3hkr_0 dataset ids: [Row(dataset_ids="['DS_tat5i46x3hkr_0']")]
# DS_dbgckv1il6v7_0 dataset ids: [Row(dataset_ids="['DS_dbgckv1il6v7_0']")]
# DS_gpsibs9f47k4_0 dataset ids: [Row(dataset_ids="['DS_5ar73nonq6l1_0', 'DS_gpsibs9f47k4_0']"),
#                                   Row(dataset_ids="['DS_gpsibs9f47k4_0']"),
#                                   Row(dataset_ids="['DS_9yr94hhj1k33_0', 'DS_5ar73nonq6l1_0', 'DS_gpsibs9f47k4_0']"),
#                                   Row(dataset_ids="['DS_gpsibs9f47k4_0', 'DS_gpsibs9f47k4_0']")]

In [ ]:
{
    "['DS_qxd7wv9yabtp_0', 'DS_la08goe2lz0g_0']": "['DS_qxd7wv9yabtp_0']",
    "['DS_q6e3bvq4y67a_0', 'DS_la08goe2lz0g_0']": "['DS_q6e3bvq4y67a_0']",
    "['DS_nublbp38wse0_0', 'DS_dhe9aqs9q1wf_0']": "['DS_nublbp38wse0_0']",
    "['DS_q6e3bvq4y67a_0', 'DS_dhe9aqs9q1wf_0']": "['DS_q6e3bvq4y67a_0']",
    "['DS_u9wd92plbetq_0', 'DS_14m394gnh3ae_0']": "['DS_u9wd92plbetq_0']",
    "['DS_5ar73nonq6l1_0', 'DS_gpsibs9f47k4_0']": "['DS_5ar73nonq6l1_0']",
    "['DS_9yr94hhj1k33_0', 'DS_5ar73nonq6l1_0', 'DS_gpsibs9f47k4_0']": "['DS_9yr94hhj1k33_0', 'DS_5ar73nonq6l1_0']",
}

In [ ]:
# Okay to remove all:
dataset_ids_to_remove_from_cos = [
    "['DS_gpsibs9f47k4_0', 'DS_gpsibs9f47k4_0']",
    "['DS_gpsibs9f47k4_0']",
]
# "['DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0']",  # Jarvis qetb
# "['DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0']",
# "['DS_e471qdt7c6db_0']",
# "['DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0', 'DS_e471qdt7c6db_0']",
# "['DS_5h3810yhu4wj_0']",  # Fe_nanoparticles_PRB_2023
# "['DS_jz1q9juw7ycj_0']",  # JARVIS_QM9_STD_JCTC
# "['DS_mvuwxu67yrdy_0']",  # JARVIS_HOPV
# "['DS_1nbddfnjxbjc_0']",  # JARVIS_SNUMAT
# "['DS_0j2smy6relq0_0']",  # NENCI-2021
# "['DS_14m394gnh3ae_0']",  # Silica_NPJCM_2022
# "['DS_tat5i46x3hkr_0']",  # JARVIS_QM9-DGL
# "['DS_dbgckv1il6v7_0']",  # JARVIS_Polymer-Genome
# ]

In [ ]:
# Next stage of removing dataset ids that were entangled
ids = [
    "DS_la08goe2lz0g_0",
    # discrepencies_and_error_metrics_NPJ_2023_vacancy_re_testing_set no pos
    # Entangled with DS_q6e3bvq4y67a_0 above
    "DS_dhe9aqs9q1wf_0",
    # discrepencies_and_error_metrics_NPJ_2023_interstitial_re_testing_set no pos
    # Entangled with DS_nublbp38wse0_0 above
    "DS_14m394gnh3ae_0",
    # Silica_NPJCM_2022 no pos
    # entangled with DS_u9wd92plbetq_0 below
    "DS_gpsibs9f47k4_0",
]
# JARVIS_Materials_Project_2020 no pos
# entangled with DS_5ar73nonq6l1_0 and DS_9yr94hhj1k33_0 below
for id in ids:
    coss.filter(sf.col("dataset_ids").contains(id)).distinct().show()

In [ ]:
cos = spark.table("ndb.colabfit.dev.co_remove_ds_ids")
cos = cos.filter(~sf.col("dataset_ids").isin(dataset_ids_to_remove_from_cos))

In [ ]:
cs = spark.table("ndb.`colabfit-prod`.prod.cs")

In [ ]:
dataset_ids_to_remove_from_cs = [
    "DS_la08goe2lz0g_0",
    # discrepencies_and_error_metrics_NPJ_2023_vacancy_re_testing_set no pos
    # Entangled with DS_q6e3bvq4y67a_0 above
    "DS_dhe9aqs9q1wf_0",
    # discrepencies_and_error_metrics_NPJ_2023_interstitial_re_testing_set no pos
    # Entangled with DS_nublbp38wse0_0 above
    "DS_gpsibs9f47k4_0",  # JARVIS_Materials_Project_2020
    "DS_e471qdt7c6db_0",  # Jarvis qe_tb
    "DS_5h3810yhu4wj_0",  # Fe_nanoparticles_PRB_2023
    "DS_jz1q9juw7ycj_0",  # JARVIS_QM9_STD_JCTC
    "DS_mvuwxu67yrdy_0",  # JARVIS_HOPV
    "DS_1nbddfnjxbjc_0",  # JARVIS_SNUMAT
    "DS_0j2smy6relq0_0",  # NENCI-2021
    "DS_14m394gnh3ae_0",  # Silica_NPJCM_2022
    "DS_tat5i46x3hkr_0",  # JARVIS_QM9-DGL
    "DS_dbgckv1il6v7_0",  # JARVIS_Polymer-Genome
]

In [ ]:
# cs.filter(sf.col("dataset_id").isin(dataset_ids_to_remove_from_cs)).select(
#     "id", "dataset_id", "name", "nconfigurations"
# ).distinct().show()
# cs.filter(sf.col("dataset_id").isin(dataset_ids_to_remove_from_cs)).select('id','dataset_id', 'name', 'nconfigurations').count()
# 136
# cs.filter(sf.col("dataset_id") == "DS_0j2smy6relq0_0").select(
#     "id", "dataset_id", "name", "nconfigurations"
# ).count()
# 131
cs = cs.filter(~sf.col("dataset_id").isin(dataset_ids_to_remove_from_cs))
cs.write.mode("overwrite").saveAsTable("ndb.colabfit.dev.cs_remove_ds_ids")

In [ ]:
sess = Session(endpoint=endpoint, access=access, secret=access)

with sess.transaction() as tx:
    table = tx.bucket("colabfit").schema("dev").table("cs_remove_ds_ids")
    table.rename("cs_remove_dataset_ids")

    rec_batch_reader = table.select(
        predicate=table.dataset_id.isin(dataset_ids_to_remove_from_cs)
    )
    rec_batch = rec_batch_reader.read_next_batch()

In [1]:
set(
    [
        "DS_la08goe2lz0g_0",
        # discrepencies_and_error_metrics_NPJ_2023_vacancy_re_testing_set no pos
        # Entangled with DS_q6e3bvq4y67a_0 above
        "DS_dhe9aqs9q1wf_0",
        # discrepencies_and_error_metrics_NPJ_2023_interstitial_re_testing_set no pos
        # Entangled with DS_nublbp38wse0_0 above
        "DS_gpsibs9f47k4_0",  # JARVIS_Materials_Project_2020
        "DS_e471qdt7c6db_0",  # Jarvis qe_tb
        "DS_5h3810yhu4wj_0",  # Fe_nanoparticles_PRB_2023
        "DS_jz1q9juw7ycj_0",  # JARVIS_QM9_STD_JCTC
        "DS_mvuwxu67yrdy_0",  # JARVIS_HOPV
        "DS_1nbddfnjxbjc_0",  # JARVIS_SNUMAT
        "DS_0j2smy6relq0_0",  # NENCI-2021
        "DS_14m394gnh3ae_0",  # Silica_NPJCM_2022
        "DS_tat5i46x3hkr_0",  # JARVIS_QM9-DGL
        "DS_dbgckv1il6v7_0",  # JARVIS_Polymer-Genome
    ]
) - set(
    [
        "DS_e471qdt7c6db_0",  # Jarvis qetb
        "DS_5h3810yhu4wj_0",  # Fe_nanoparticles_PRB_2023
        "DS_jz1q9juw7ycj_0",  # JARVIS_QM9_STD_JCTC
        "DS_mvuwxu67yrdy_0",  # JARVIS_HOPV
        "DS_1nbddfnjxbjc_0",  # JARVIS_SNUMAT
        "DS_0j2smy6relq0_0",  # NENCI-2021
        "DS_14m394gnh3ae_0",  # Silica_NPJCM_2022
        "DS_tat5i46x3hkr_0",  # JARVIS_QM9-DGL
        "DS_dbgckv1il6v7_0",  # JARVIS_Polymer-Genome
    ]
)

{'DS_dhe9aqs9q1wf_0', 'DS_gpsibs9f47k4_0', 'DS_la08goe2lz0g_0'}